In [1]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()



0

In [3]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET



In [4]:
processed_data_dir_path = "/workspace/data_processed"

In [5]:
import glob 
data_open_slr_paths = glob.glob("/workspace/data/openSLRData/*")

In [6]:
data_open_slr_paths

['/workspace/data/openSLRData/2',
 '/workspace/data/openSLRData/7',
 '/workspace/data/openSLRData/3',
 '/workspace/data/openSLRData/4',
 '/workspace/data/openSLRData/5',
 '/workspace/data/openSLRData/8',
 '/workspace/data/openSLRData/6',
 '/workspace/data/openSLRData/1']

In [7]:
# Creating Processor
data_1 = '/workspace/data/openSLRData/8'

In [8]:
transcript_json = data_1 + "/" +  "data.json"
print(transcript_json)
data = pd.read_json(transcript_json)

/workspace/data/openSLRData/8/data.json


In [9]:
data['audioFilename']

0       00133a9876.wav
1       0022362ec0.wav
2       002a8d9076.wav
3       002ae35cd3.wav
4       002bb5d867.wav
             ...      
6621    ffd1373b55.wav
6622    ffd3a15d9b.wav
6623    ffe8444fbb.wav
6624    fff0853ee3.wav
6625    fff7293e3b.wav
Name: audioFilename, Length: 6626, dtype: object

In [10]:
for i in range(len(data)):
    file = data_1  + "/" + data['audioFilename'][i]
    data['audioFilename'][i] = file

In [11]:
data['audioFilename']

0       /workspace/data/openSLRData/8/00133a9876.wav
1       /workspace/data/openSLRData/8/0022362ec0.wav
2       /workspace/data/openSLRData/8/002a8d9076.wav
3       /workspace/data/openSLRData/8/002ae35cd3.wav
4       /workspace/data/openSLRData/8/002bb5d867.wav
                            ...                     
6621    /workspace/data/openSLRData/8/ffd1373b55.wav
6622    /workspace/data/openSLRData/8/ffd3a15d9b.wav
6623    /workspace/data/openSLRData/8/ffe8444fbb.wav
6624    /workspace/data/openSLRData/8/fff0853ee3.wav
6625    /workspace/data/openSLRData/8/fff7293e3b.wav
Name: audioFilename, Length: 6626, dtype: object

In [12]:
data.rename(columns = {'text':'sentence'}, inplace = True)
data.rename(columns = {'audioFilename':'path'}, inplace = True)

In [13]:
len(data)

6626

In [14]:
train = data[:]
len(train)


6626

In [15]:
common_voice_train_1 = Dataset.from_pandas(train[:])
# common_voice_train_2 = Dataset.from_pandas(train[5000:])

print(len(common_voice_train_1))
# print(len(common_voice_train_2))

6626


In [16]:
print(common_voice_train_1['sentence'][3])
# print(common_voice_train_2['sentence'][3])

सन्दर्भमा कुरा गर्दा


In [17]:
common_voice_train_1 = common_voice_train_1.remove_columns([ "collectionSource","snr",  "duration", "gender"])
# common_voice_train_2 = common_voice_train_2.remove_columns(["collectionSource","snr",  "duration", "gender"])


In [18]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [19]:
common_voice_train_1 = common_voice_train_1.map(remove_special_characters)
# common_voice_train_2 = common_voice_train_2.map(remove_special_characters)

In [20]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [21]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)

In [22]:
# common_voice_train_2 = common_voice_train_2.map(speech_file_to_array_fn, remove_columns=common_voice_train_2.column_names,num_proc=4)

In [22]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train_1)-1)

ipd.Audio(data=np.asarray(common_voice_train_1[rand_int]["speech"]), autoplay=True, rate=16000)

In [23]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [24]:
tokenizer = Wav2Vec2CTCTokenizer("/workspace/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [25]:
len(processor.tokenizer)

62

In [26]:
common_voice_train_1 = common_voice_train_1.map(prepare_dataset, remove_columns=common_voice_train_1.column_names, batch_size=8, num_proc=4, batched=True)

In [28]:
# common_voice_train_2 = common_voice_train_2.map(prepare_dataset, remove_columns=common_voice_train_2.column_names, batch_size=8, num_proc=4, batched=True)

In [27]:
# common_voice_train_2

In [30]:
# common_voice_train = concatenate_datasets([common_voice_train_1, common_voice_train_2])

In [28]:
processed_data_dir_path_ = processed_data_dir_path + "/" + str(15)

In [29]:
processed_data_dir_path_

'/workspace/data_processed/15'

In [30]:
common_voice_train_1.save_to_disk(processed_data_dir_path_)

# .arrrow -> (speech array(....), target_text = [1, 2, 3 ,4]) 